# MLP with PyTorch

At this point, I know what MLPs are. We built one from scratch one of the previous folder. In this notebook, I want to implement an MLP again, but using PyTorch. This MLP will trained and evaluated on the MNIST dataset. 

# The MNIST dataset

Conveniently, the MNIST dataset is provided in PyTorch through the `torchvision` module, specifically through the `torchvision.dataset` module.

In the following cell, I import the `torchvision` and `transforms` modules. The second module, as the name suggests, let us perform **common transformations on image data**. According to the [documentation](https://pytroch.org/vision/stable/transforms.html), Transforms are common image transformations available in the `torchvision.transforms` module.

Another interesting feature is that transform operations can be **chained** together using `Compose`. We will use it in a couple of cells below.

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

With the modules loaded, I want to load the dataset itself, and specify hyperparameters such as the size of the training and testing sets, and size of the mini-batches.

In [2]:
image_path = './'

transform = transforms.Compose([
    transforms.ToTensor()
])

mnist_train_dataset = torchvision.datasets.MNIST(
    root=image_path, train=True,
    transform=transform, download=True 
)

mnist_test_dataset = torchvision.datasets.MNIST(
    root=image_path, train=False,
    transform=transform, download=True  
)

batch_size = 64
torch.manual_seed(1)

Okay, what just happened? Since I want to download a dataset I created a `image_path` variable to store the path where I would like images to be stored, should they be downloaded or read from the filesystem, if I do not want the dataset to be downloaded.

Then I move on create a `transform` pipeline. Ours only has one operation: `transform.ToTensor()`. The `ToTensor()` method (1)converts the pixel features into a floating type tensor and (2) normalizes the pixel from range [0, 255] to range [0, 1].

After that is where I effectively create the training and testing dataset using the MNIST dataset. Since, I do not have it on my machine, I asked PyTorch to download it for me using the `download` paramater. I also want PyTorch to perform the `transform` we created earlier on the images being downloaded. I specify which operation to perform using the `transform` paramater.

I finish with specifying the batch size, and manually setting the seed number of random number generation.

With that being done, we cannot use the dataset just yet. We must pass the `Dataset` objects (`mnist_train_dataset` and `mnist_test_dataset`) into a dataset a `DataLoader` object. Remember through a `DataLoader`, we can properly iterate over a given dataset. Okay, let's do it:

In [3]:
from torch.utils.data import DataLoader
train_dl = DataLoader(mnist_train_dataset,
                      batch_size, shuffle=True)

We successfully created the data loader, with batches of 64 samples. Let's move on :)

# Building the Model

This section of the notebook is concerned with building the MLP to classify digits from the dataset we downloaded earlier.

Our MLP will have:

- an input layer
- a hidden layer (32 activation units)
- a hidden layer (16 activation units)
- an output layer

Let's define the above layers in code

In [4]:
hidden_units = [32, 16] #number of activation units in EACH hidden layer
image_size = mnist_train_dataset[0][0].shape #mnist_train_dataset[0] is a tuple (image[tensor], label)
input_size = image_size[0] * image_size[1] * image_size[2] #number of channels * image height * image width

# all the layers in the network 
all_layers = [nn.Flatten()]
for hidden_unit  in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 10)) #output layer

We successfully created all the layers in the network, and we stored them all into the an array called: `all_layers`.

Let's now create a model containing the layers we just created.

In [5]:
model = nn.Sequential(*all_layers)

That's it. Done! Since each layer comes one after the other in an MLP, we use the `torch.nn.Sequential` module to place those layers *sequentially*.

Let's print the model and see all the layers we inserted.

In [6]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

When an image, represented by a $(1 \times 28 \times 28)$ vector, gets fed into the network  the `Flatten` layer flattens it to a $(1 \times 784)$ vector.

This flattened vector goes through the first `Linear` layer. This is the **input** layer. It turns the $(1 \times 784)$ to a $(1 \times 32)$ vector. `(1)`

Right after that, the vector goes through a `RELU` activation function. The size of the vector remains the same: $(1 \times 32)$

The $(1 \times 32)$ vector goes through a second `Linear` layer, which downsizes it to a $(1 \times 16)$ vector. This vector also goes through, a `RELU` activation.

Lastly, the $(1 \times 16)$ vector goes through the last `Linear` layer. The output layer. Which turns the $(1 \times 16)$ matrix into a $(1 \times 10)$ vector. Which is our prediction.

Let's now train the model

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_dl:
        #compute forward pass and loss value
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        
        #compute gradients through backpropagation
        loss.backward()

        #update weights based on gradients, then reset the gradients to zero 
        optimizer.step()
        optimizer.zero_grad()

        #how correct was our model on this batch?
        is_correct = (
            torch.argmax(pred, dim=1) == y_batch
        ).float()

        #Number of times model was correct on this batch
        accuracy_hist_train += is_correct.sum()
    
    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch} Accuracy: '
          f'{accuracy_hist_train:.4f}')

Epoch 0, Accuracy: 0.8514
Epoch 1, Accuracy: 0.9287
Epoch 2, Accuracy: 0.9422
Epoch 3, Accuracy: 0.9492
Epoch 4, Accuracy: 0.9538
Epoch 5, Accuracy: 0.9584
Epoch 6, Accuracy: 0.9622
Epoch 7, Accuracy: 0.9647
Epoch 8, Accuracy: 0.9673
Epoch 9, Accuracy: 0.9688
Epoch 10, Accuracy: 0.9710
Epoch 11, Accuracy: 0.9720
Epoch 12, Accuracy: 0.9737
Epoch 13, Accuracy: 0.9750
Epoch 14, Accuracy: 0.9769
Epoch 15, Accuracy: 0.9780
Epoch 16, Accuracy: 0.9788
Epoch 17, Accuracy: 0.9799
Epoch 18, Accuracy: 0.9813
Epoch 19, Accuracy: 0.9816


Cool, our model finished training. The model acheived a 9..% accuracy on training set, but let's see how well it performs on the testing set which is data it has never seen before.

In [9]:
pred = model(mnist_test_dataset.data.float())

is_correct = (
    torch.argmax(pred, dim=1) == mnist_test_dataset.targets
).float()

print(f'Accuracy on Test set: {is_correct.mean():.4f}')

Accuracy on Test set: 0.9605


That's it!

We successfully trained and evaluated our model on the MNIST dataset. The performance on the training set is 2% higher, than the performance on the testing set. I think the model generalizes fairly well.

#  Potential Experiments

Here are few ideas to further the exploration:

1. Feed foward only ONE training example through the network. What is the shape of the output vector? Can you also print its values?

2. We used `RELU` as our activation function. Modify the model so it uses sigmoid instead. Does it affect the model performance on the testing set?

3. Using the model we built, can you reset it and graph the weights as the model learns. See how they change over time? Might be a cool visual.